In [2]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from bs4 import BeautifulSoup

import time
from loadbar import LoadBar

In [2]:
inicio=10
valores=[]
while inicio != 24:
    
    fin=inicio+1
    valores.append(str(inicio)+str(fin))
    inicio +=1

In [5]:
bar = LoadBar(
    max=5
)
bar.start()
for x in range(5):
    bar.update(step=x)
    time.sleep(0.5)
bar.end()

5/5 (100%) [....................] Time 0:00:02


### General Statistics

In [5]:
def filtrarColumnas(data):
    data=data.drop(data.columns[range(data.columns.get_loc("B365H"),data.shape[1])],axis=1)
    data= data.drop(columns=["Referee" ],axis=1)

    if "Time" in data:
        data = data.drop(columns=["Time"],axis=1)
    return data

    

In [6]:
def crearMatriz(data):
    tmp_matrix = np.full((20, 40), np.nan)
    columnas =  [f'Jornada_{i}' for i in range(1, 41)]
    filas = sorted(data['HomeTeam'].unique())
    tmp_matrix = pd.DataFrame(tmp_matrix, columns=columnas)
    filas = filas
    tmp_matrix.index = filas
    tmp_matrix
    return tmp_matrix, filas

In [7]:
def goalDiff(data,filas,goal_diff):
    for j in filas:
        t=1
        #for i in range(data.shape[0] - 1, -1, -1):
        for i in range(data.shape[0]):
            if j == data.iloc[i, 2]:
                tmp_diff = data.iloc[i, 4] - data.iloc[i, 5]
                if pd.isna(goal_diff.loc[j, 'Jornada_1']):
                    goal_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    goal_diff.loc[j, f'Jornada_{t}'] = goal_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1  
            
            elif j == data.iloc[i, 3]:
                tmp_diff = data.iloc[i, 5] - data.iloc[i, 4]
                if pd.isna(goal_diff.loc[j, 'Jornada_1']):
                    goal_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    goal_diff.loc[j, f'Jornada_{t}'] = goal_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1
    
    
    goal_diff = goal_diff / goal_diff.max().max()
    goal_diff.insert(loc=0,column="Jornada_0",value=0)
    goal_diff.loc[pd.isna(goal_diff["Jornada_38"]),"Jornada_38"] = goal_diff.loc[pd.isna(goal_diff["Jornada_38"]),"Jornada_37"]

    return goal_diff, filas

In [8]:
def cornersDiff(data,filas,corns_diff):
    for j in filas:
        t=1
        #for i in range(data.shape[0] - 1, -1, -1):
        for i in range(data.shape[0]):
            if j == data.iloc[i, 2]:
                tmp_diff = data.iloc[i, 16] - data.iloc[i, 17]
                if pd.isna(corns_diff.loc[j, 'Jornada_1']):
                    corns_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    corns_diff.loc[j, f'Jornada_{t}'] = corns_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1  

            elif j == data.iloc[i, 3]:
                tmp_diff = data.iloc[i, 17] - data.iloc[i, 16]
                if pd.isna(corns_diff.loc[j, 'Jornada_1']):
                    corns_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    corns_diff.loc[j, f'Jornada_{t}'] = corns_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1
                    
    corns_diff= corns_diff/ corns_diff.max().max()
    corns_diff.insert(loc=0,column="Jornada_0",value=0)
    corns_diff.loc[pd.isna(corns_diff["Jornada_38"]),"Jornada_38"] = corns_diff.loc[pd.isna(corns_diff["Jornada_38"]),"Jornada_37"]

    return corns_diff, filas


In [9]:
def shootsDiff(data,filas,shoots_diff):
    for j in filas:
        t=1
        #for i in range(data.shape[0] - 1, -1, -1):
        for i in range(data.shape[0]):
            if j == data.iloc[i, 2]:
                tmp_diff = data.iloc[i, 12] - data.iloc[i, 13]
                if pd.isna(shoots_diff.loc[j, 'Jornada_1']):
                    shoots_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    shoots_diff.loc[j, f'Jornada_{t}'] = shoots_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1  

            elif j == data.iloc[i, 3]:
                tmp_diff = data.iloc[i, 13] - data.iloc[i, 12]
                if pd.isna(shoots_diff.loc[j, 'Jornada_1']):
                    shoots_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    shoots_diff.loc[j, f'Jornada_{t}'] = shoots_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1
                    
    shoots_diff= shoots_diff/ shoots_diff.max().max()
    shoots_diff.insert(loc=0,column="Jornada_0",value=0)
    shoots_diff.loc[pd.isna(shoots_diff["Jornada_38"]),"Jornada_38"] = shoots_diff.loc[pd.isna(shoots_diff["Jornada_38"]),"Jornada_37"]

    return shoots_diff, filas    

In [10]:
def matrizVacia(data):
    cols = data.columns[[4, 5] + list(range(7, 22))]
    data=data.drop(cols,axis=1)
    data['HTCD'] = None
    data['ATCD'] = None
    data['HTGD'] = None
    data['ATGD'] = None
    data['HTSD'] = None
    data['ATSD'] = None
    data['HomeRating(ELO) 0.05'] = None
    data['AwayRating(ELO) 0.05'] = None
    data['HomeRating(ELO) 0.1'] = None
    data['AwayRating(ELO) 0.1'] = None
    data['HomeRating(ELO) 0.25'] = None
    data['AwayRating(ELO) 0.25'] = None
    data['HomeRating(ELO) 0.4'] = None
    data['AwayRating(ELO) 0.4'] = None
    data['HomeRating(ELO) 0.55'] = None
    data['AwayRating(ELO) 0.55'] = None
    data['HomeRating(ELO) 0.7'] = None
    data['AwayRating(ELO) 0.7'] = None
    data['HomeRating(ELO) 0.85'] = None
    data['AwayRating(ELO) 0.85'] = None
    data['HomeRating(ELO) 0.9'] = None
    data['AwayRating(ELO) 0.9'] = None
    data['Value Diff'] = None
    data['Attack Diff'] = None
    data['Mid Game Diff'] = None
    data['Defense Diff'] = None
    data['Over Game Diff'] = None
    return data

In [11]:
def resultadosCorner(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,"HTCD"]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,"ATCD"]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas                                           

In [12]:
def resultadosShoots(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,"HTSD"]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,"ATSD"]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas

In [13]:
def resultadosGoles(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,"HTGD"]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,"ATGD"]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas

In [14]:
def ratingsMatrix(data):
    tmp_matrix = np.full((20, 40), np.nan)
    columnas =  [f"{i}" for i in range(1, 41)]
    filas = sorted(data['HomeTeam'].unique())
    tmp_matrix = pd.DataFrame(tmp_matrix, columns=columnas)
    filas = filas
    tmp_matrix.index = filas
    tmp_matrix.insert(loc=0,column="0",value=1)
    return tmp_matrix, filas

In [15]:
def ratingDiff(data,rating_diff,gamma):
    for i in range(data.shape[0]):
        id1=data.iloc[i,2]
        id2=data.iloc[i,3]
        posit1=rating_diff.loc[id1,:].last_valid_index()
        posit2=rating_diff.loc[id2,:].last_valid_index()

        next_posit1= rating_diff.columns.get_loc(str(int(posit1)+1))
        next_posit2= rating_diff.columns.get_loc(str(int(posit2)+1))

        mu_alfa = rating_diff.loc[id1,str(posit1)]
        mu_beta = rating_diff.loc[id2,str(posit2)]

        if data.iloc[i,6] == "H":
            mu_alfa = mu_alfa + gamma * mu_beta
            mu_beta = mu_beta - gamma * mu_alfa

        elif data.iloc[i,6] == "A":
            mu_alfa = mu_alfa - gamma * mu_beta
            mu_beta = mu_beta + gamma * mu_alfa

        else:
            mu_alfa = mu_alfa - gamma* (mu_alfa - mu_beta)
            mu_beta = mu_beta - gamma* (mu_beta - mu_alfa)

        rating_diff.loc[id1,str(next_posit1)] = mu_alfa
        rating_diff.loc[id2,str(next_posit2)] = mu_beta
        
    columnas =  [f"Jornada_{i}" for i in range(0, 41)]
    rating_diff.columns = columnas
        
    return rating_diff

In [16]:
def resultadosRating(data,filas,variable,resultados,colA,colH):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,colH]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,colA]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas

In [17]:
gammas = [0.05,0.1,0.25,0.4,0.55,0.7,0.85,0.9]

## **Team's Market Value**

In [18]:
def extract_club_values_table(soup):
    # Buscar la tabla en el objeto BeautifulSoup proporcionado
    table = soup.find('div', class_='responsive-table').find('table', class_='items')
    
    if table:
        # Convertir la tabla HTML a DataFrame de pandas
        df = pd.read_html(str(table))[0]
        df = df.iloc[:20,[1,6]]     
        return df

In [19]:
dfs = []

# Iterar a través de los años en el rango deseado
for i in range(10, 24):
    url = f'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=20{i}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        df = extract_club_values_table(soup)
        
        if df is not None:
            print(f"Tabla encontrada para el año 20{i}")
            dfs.append(df)# Agregar el dataframe a la lista de dataframe



Tabla encontrada para el año 2010
Tabla encontrada para el año 2011
Tabla encontrada para el año 2012
Tabla encontrada para el año 2013
Tabla encontrada para el año 2014
Tabla encontrada para el año 2015
Tabla encontrada para el año 2016
Tabla encontrada para el año 2017
Tabla encontrada para el año 2018
Tabla encontrada para el año 2019
Tabla encontrada para el año 2020
Tabla encontrada para el año 2021
Tabla encontrada para el año 2022
Tabla encontrada para el año 2023


### **Formateado de los datos**

In [20]:
def convertir_valor_df(valor):
    if valor.endswith("bn"):
        numero = float(valor[1:-2]) * 1_000_000_000
    elif valor.endswith("m"):
        numero = float(valor[1:-1]) * 1_000_000
    else:
        numero = valor
    return f"{numero:.0f}"

In [21]:
%run IdentificadoresPremier.ipynb
for i in range(len(dfs)):
    
    dfs[i]["Club.1"]=rename_market_value_premier(dfs[i]["Club.1"])
    dfs[i].columns = ["Club","Value"]
    dfs[i]["Value"] = dfs[i]["Value"].apply(convertir_valor_df)
    dfs[i]["Value"] = pd.to_numeric(dfs[i]["Value"])

In [22]:
lista=[11,"12_8","13_11","14_12","15_16","16_26","17_75","18_175","19_280","20_358","21_421","22_489","23_558","24_593"]

### **FIFA'S STATISTICS**

In [23]:
%run IdentificadoresPremier.ipynb
fifa_statistics=[]
for i in lista:
    url = f"https://www.fifaindex.com/teams/fifa{i}/?league=13&order=desc"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find("table", class_="table-teams")
    
    # Extraer los encabezados de las columnas
    headers = [header.text.strip() for header in table.find_all("th")]
    
    # Extraer las filas de la tabla
    rows = []
    for row in table.find_all("tr")[1:]:  # Excluyendo el encabezado
        columns = row.find_all("td")
        if columns:
            rows.append([col.text.strip() for col in columns])
    
    # Crear el DataFrame de Pandas
    df = pd.DataFrame(rows, columns=headers)
    df= df.dropna()
    df = df.drop("League",axis=1)

    df["Name"] = rename_fifa_stats_premier(df["Name"])
    fifa_statistics.append(df)

## **Obtención de estadísticas en la tabla final**

In [24]:
final_results=pd.DataFrame()
year=0
for i in valores:
    print(i)
    #Obtener la tabla con la que vamos a trabajar:
    url = f"https://www.football-data.co.uk/mmz4281/{i}/E0.csv"
    s = requests.get(url).content.decode('utf-8') 
    data=StringIO(s)
    tabla = pd.read_csv(data)
    
    #renombramos los nombres en "HomeTeam" "AwayTeam" por sus identificadores
    %run IdentificadoresPremier.ipynb
    tabla["HomeTeam"] = rename_football_data_co_uk_premier(tabla["HomeTeam"])
    tabla["AwayTeam"] = rename_football_data_co_uk_premier(tabla["AwayTeam"])
    tabla = tabla.dropna(how="all")
    #tabla = renombrarColumnas(tabla)
    tabla = filtrarColumnas(tabla)
    
    #Creamos las matrices temporales para las diferentes variables
    tmp_goal_diff, filas  = crearMatriz(tabla)
    tmp_corns_diff, filas = crearMatriz(tabla)
    tmp_shoots_diff, filas = crearMatriz(tabla)
    
    #Diferencia de goles
    goal_diff, filas =goalDiff(tabla,filas,tmp_goal_diff)
    corns_diff, filas = cornersDiff(tabla,filas,tmp_corns_diff)
    shoots_diff, filas = shootsDiff(tabla,filas,tmp_shoots_diff)

    #Generamos la matriz de resultados
    resultados=matrizVacia(tabla)
    
    #Añadimos los resultados a las columnas de corners
    resultados, filas= resultadosCorner(tabla,filas,corns_diff,resultados)
    resultados, filas= resultadosShoots(tabla,filas,shoots_diff,resultados)
    resultados, filas = resultadosGoles(tabla,filas,goal_diff, resultados)

 
    #Crear matrix de ratings (Sistema ELO)
    for gamma in gammas:
        rating_diff, filas =ratingsMatrix(tabla)
        rating_diff = ratingDiff(tabla,rating_diff,gamma=gamma)
        colA=f'AwayRating(ELO) {gamma}'
        colH=f'HomeRating(ELO) {gamma}'
        
        resultados, filas = resultadosRating(tabla,filas,rating_diff,resultados,colA,colH)
    #final_results= pd.concat([final_results,resultados])

    for w in range(resultados.shape[0]):
        for j in range(dfs[1].shape[0]):
            clubH = resultados.iloc[w,2]
            clubA = resultados.iloc[w,3]
            valorH= dfs[year].loc[dfs[year]["Club"] == clubH,"Value"].item()
            valorA= dfs[year].loc[dfs[year]["Club"] == clubA,"Value"].item()
            resultados.iloc[w,27] = valorH - valorA

    for r in range(resultados.shape[0]):
        for j in range(fifa_statistics[1].shape[0]):
            clubH = resultados.iloc[r,2]
            clubA = resultados.iloc[r,3]
            attH= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubH,"ATT"].item()
            attA= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubA,"ATT"].item()
            midH= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubH,"MID"].item()
            midA= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubA,"MID"].item()
            defH= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubH,"DEF"].item()
            defA= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubA,"DEF"].item()
            ovrH= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubH,"OVR"].item()
            ovrA= fifa_statistics[year].loc[fifa_statistics[year]["Name"] == clubA,"OVR"].item()
            resultados.iloc[r,28] = int(attH) - int(attA)
            resultados.iloc[r,29] = int(midH) - int(midA)
            resultados.iloc[r,30] = int(defH) - int(defA)
            resultados.iloc[r,31] = int(ovrH) - int(ovrA)

    year +=1

    final_results= pd.concat([final_results,resultados])
    print(f"Fin de la iteracion: {year}/14")
#final_results.iloc[:3040, 1] = final_results.iloc[:3040, 1].str[:6] + "20" + final_results.iloc[:3040, 1].str[6:]
#final_results.iloc[3040:, 1] = pd.to_datetime(final_results.iloc[3040:, 1], format='%d/%m/%Y')
#final_results['Date'] = pd.to_datetime(final_results['Date'], infer_datetime_format=True)"""
final_results.iloc[:1900, 1] = final_results.iloc[:1900, 1].str[:6] + "20" + final_results.iloc[:1900, 1].str[6:]
final_results.iloc[2280:2660, 1] = final_results.iloc[2280:2660, 1].str[:6] + "20" + final_results.iloc[2280:2660, 1].str[6:]

1011
Fin de la iteracion: 1/14
1112
Fin de la iteracion: 2/14
1213
Fin de la iteracion: 3/14
1314
Fin de la iteracion: 4/14
1415
Fin de la iteracion: 5/14
1516
Fin de la iteracion: 6/14
1617
Fin de la iteracion: 7/14
1718
Fin de la iteracion: 8/14
1819
Fin de la iteracion: 9/14
1920
Fin de la iteracion: 10/14
2021
Fin de la iteracion: 11/14
2122
Fin de la iteracion: 12/14
2223
Fin de la iteracion: 13/14
2324
Fin de la iteracion: 14/14


Tras comprobar manualmente los resultados desde la página web, se verifica la veracidad de los resultadoss

#### **Extracción Nombre Árbitro del partido**

In [60]:
def extraer_html(url):
    url = str(url)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [61]:
%run IdentificadoresPremier.ipynb
referees=[]
for i in range(10,24):
    url = f'https://www.bdfutbol.com/es/t/teng20{i}-{i+1}.html?tab=results&jornada=-'
    soup=extraer_html(url)
    tables = soup.find_all("table")
    def find_matching_table(tables):
        for table in tables:
            headers = [header.get_text(strip=True) for header in table.find_all("th")]
            # Criterios de búsqueda simplificados para el ejemplo
            if "Fecha" in headers and "Local" in headers and "Visitante" in headers and "Árbitro" in headers:
                return table
        return None
    
    # Encontrar la tabla
    matching_table = find_matching_table(tables)
    
    # Extraeremos los datos de la tabla coincidente si se encontró una
    if matching_table:
        rows = matching_table.find_all("tr")
        table_data = []
        headers = [header.get_text(strip=True) for header in rows[0].find_all("th")]
    
        for row in rows[1:]:  # Saltar la fila de encabezados
            cols = row.find_all(["td", "th"])
            cols = [ele.text.strip() for ele in cols]
            table_data.append(cols)
        
        result = {
            "headers": headers,
            "data": table_data
        }
    
    lista =list(result.values())[1][1:]
    lista_filtrada = [sublista for sublista in lista if len(sublista) > 1]
    data =  pd.DataFrame(lista_filtrada, columns=['Date', 'HomeTeam','Resultado','AwayTeam','Estadio' ,'Arbitro'])
    #data.replace('',pd.NA,inplace=True)
    data.dropna(inplace=True)
    data["HomeTeam"]= rename_fifa_stats_premier(data["HomeTeam"])
    data["AwayTeam"]= rename_fifa_stats_premier(data["AwayTeam"])
    #data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
    referees.append(data)

referees_df = pd.concat(referees)
referees_df["Arbitro"].replace('',pd.NA,inplace=True)
referees_df["HomeTeam"].replace('Local',pd.NA,inplace=True)
referees_df.reset_index(inplace=True)
rows_to_drop=referees_df.loc[pd.isna(referees_df["Arbitro"]),:].index
headers_to_drop = referees_df.loc[pd.isna(referees_df["HomeTeam"]),:].index
referees_df.drop(index=headers_to_drop,inplace=True)
referees_df.drop(index=rows_to_drop,inplace=True)

In [62]:
referees_df["HomeTeam"].replace('Local',pd.NA,inplace=True)

In [63]:
referees_df.loc[pd.isna(referees_df["HomeTeam"]),:].index

Index([], dtype='int64')

In [64]:
referees_df.shape

(5233, 7)

#### **Extracción Identificadores de los respectivos árbitros**

In [65]:
referees_id =[]
for i in range(10,24):
    url = f'https://www.bdfutbol.com/es/t/teng20{i}-{i+1}.html?tab=referees'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    links = soup.find_all('a', href=True)
    
    # Extrayendo los parámetros y nombres de los árbitros
    data = []
    for link in links:
        href = link.get('href')
        if 'r/r' in href:
            # Extrayendo el parámetro numérico del enlace
            param = href.split('/')[-1].replace('.html', '')
            # Extrayendo el nombre del árbitro
            name = link.text.strip()
            data.append({'Parametro': param, 'Arbitro': name})
    
    # Creando un DataFrame de pandas con los datos recopilados
    df = pd.DataFrame(data)
    referees_id.append(df)

referees_id = pd.concat(referees_id)
referees_id = referees_id[referees_id["Parametro"].apply(len) >3].drop_duplicates()

In [66]:
referees = pd.merge(referees_df,referees_id, on ="Arbitro",how="outer",indicator=True)

#### **Extracción estadísticas del árbitro**

In [67]:
ref_ids=list(referees_id["Parametro"].unique())

In [68]:
ref_stats = pd.DataFrame(data= None)
for i in ref_ids:
    url = f'https://www.bdfutbol.com/es/r/{i}.html?p=stats'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    tables = soup.find_all('table')
    target_table = None
    for table in tables:
        if "Expulsiones Por Partido" in table.text and "Tarjetas Amarillas Por Partido" in table.text:
            target_table = table
            break
    
    # Si se encontró la tabla, convertirla en un DataFrame
    if target_table:
        df = pd.read_html(str(target_table))[0]  # pd.read_html devuelve una lista de DataFrames
        df = df.iloc[[1, 2, 3,4, 7,8], [1]]  # Selecciona solo las filas y columnas deseadas
        df.index = [1, 2, 3,4, 7,8]  # Actualiza el índice para coincidir con ref_stats
        df.columns = [i]  # Renombra la columna con el id del árbitro
    else:
        df = pd.DataFrame(index=[1, 2, 3, 6, 7], columns=[i])  # Crea un DataFrame vacío con el índice correcto si no se encuentra la tabla
    
    ref_stats = pd.concat([ref_stats, df], axis=1)  # Concatena horizontalmente

In [69]:
ref_stats

,r611300,r611324,r611314,r611321,r611301,r611308,r611326,r611311,r611309,r611323,...,r612019,r612023,r602015,r602154,r610095,r602308,r610436,r610435,r620046,r610440
1,561(#1),308(#7),195(#26),291(#9),364(#5),391(#3),281(#11),181(#29),463(#2),262(#17),...,40(#115),14,16(#196),8,5,14,2,2,2,1
2,248(#1),149(#6),92(#26),137(#9),158(#5),167(#3),124(#15),88(#28),214(#2),131(#12),...,17(#123),7(#211),8(#201),3(#250),3,7(#207),1,0,1,0
3,154(#1),81(#11),51(#29),78(#16),119(#4),126(#3),93(#8),51(#28),145(#2),71(#19),...,13(#89),3(#210),2(#220),3(#202),0,4(#174),1,1,1,0
4,159(#1),78(#9),52(#30),76(#10),87(#5),98(#3),64(#15),42(#36),104(#2),60(#20),...,10(#124),4(#191),6(#161),2(#237),2(#235),3(#211),0,1,0,1
7,"3,68(#18)","3,51(#26)","3,66(#20)","3,27(#42)","3,55(#25)","3,17(#50)","2,22(#80)","2,48(#74)","3,27(#44)","2,72(#69)",...,"4,47(#3)","3,43(#29)","4,12(#7)","4,38(#4)","3,8(#14)","3,07(#57)",4(#10),4(#9),"5,5(#1)",5(#2)
8,"0,2(#31)","0,22(#28)","0,13(#69)","0,16(#46)","0,14(#60)","0,16(#49)","0,13(#72)","0,17(#43)","0,15(#57)","0,15(#58)",...,"0,12(#76)","0,07(#135)","0,12(#77)","0,38(#8)","0,4(#6)","0,07(#136)",0,"0,5(#3)","0,5(#2)",0


In [70]:
ref_stats = ref_stats.T

In [71]:
ref_stats.rename(columns={1:"Partidos",2:"HomeWin",3:"AwayWin",4:"Empates",7:"Yellow Card/Match",8:"Red Card/Match"},inplace=True)

In [72]:
for column in ref_stats.columns:
    ref_stats[column] = ref_stats[column].str.replace(r'\s*\([^)]*\)', '', regex=True)

In [73]:
ref_stats

,Partidos,HomeWin,AwayWin,Empates,Yellow Card/Match,Red Card/Match
r611300,561,248,154,159,"3,68","0,2"
r611324,308,149,81,78,"3,51","0,22"
r611314,195,92,51,52,"3,66","0,13"
r611321,291,137,78,76,"3,27","0,16"
r611301,364,158,119,87,"3,55","0,14"
r611308,391,167,126,98,"3,17","0,16"
r611326,281,124,93,64,"2,22","0,13"
r611311,181,88,51,42,"2,48","0,17"
r611309,463,214,145,104,"3,27","0,15"
r611323,262,131,71,60,"2,72","0,15"


In [74]:
ref_stats["% HomeWin"] = ref_stats["HomeWin"].astype("int") /  ref_stats["Partidos"].astype("int")
ref_stats["% AwayWin"] = ref_stats["AwayWin"].astype("int") /  ref_stats["Partidos"].astype("int")
ref_stats["% DrawWin"] = ref_stats["Empates"].astype("int") /  ref_stats["Partidos"].astype("int")


In [75]:
ref_stats = ref_stats.reset_index()

In [76]:
ref_stats.rename(columns={"index":"Parametro"},inplace=True)

In [77]:
final_referees = pd.merge(referees,ref_stats,on="Parametro")

In [78]:
final_refs = final_referees.loc[:,["Date","HomeTeam","AwayTeam","Estadio","Arbitro","Parametro","Yellow Card/Match","Red Card/Match","% HomeWin","% AwayWin","% DrawWin"]]

In [92]:
Premier = pd.merge(final_results, final_refs, on=["Date", "HomeTeam", "AwayTeam"], how="outer", indicator=True)

In [93]:
Premier = Premier[Premier["_merge"] == "both"]

In [94]:
Premier.shape

(5233, 41)

In [83]:
Premier.to_csv("./Premier.csv")